In [1]:
import pandas as pd
import sys
import os
import string
import numpy as np
import scipy.stats as stats
import re

In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):    
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [3]:
def calc_mean(list):
    mean = np.mean(list)
    return mean/1000
    # if len(list) == 1:
    #     return list[0]
    # else:    
    #     mean = np.mean(list)        
        # std = np.std(list)
        # normal_list = []
        # for l in list:
        #     z = (l-mean)/std
        #     if z <= 1 and z>= -1:
        #         normal_list.append(l)
        # avg_value = np.mean(normal_list) /1000 #convert milisecond to second   
        # return avg_value 

In [4]:
def merge_results(paths):
    dfs = []
    base_index = 0
    for path in paths:        
        data = pd.read_csv( path, sep=',')
        df_index = [ *range(base_index, base_index + len(data), 1)]
        base_index += len(data)
        data.index=df_index
        dfs.append(data)


    result =  pd.concat(dfs)
    return result

In [5]:
def results():
    root_path = '../archive'
    write_results = [f'{root_path}/read_write/CPP_R13_Experiment1_Write_times.dat', 
                     f'{root_path}/read_write/Java_R1_Experiment1_Write_times.dat',
                     f'{root_path}/read_write/Java_R2_Experiment1_Write_times.dat',
                     f'{root_path}/read_write/Java_R3_Experiment1_Write_times.dat',
                     f'{root_path}/read_write/Rust_R13_Experiment1_Write_times.dat']

    read_results = [f'{root_path}/read_write/CPP_R13_Experiment2_Read_times.dat', 
                    f'{root_path}/read_write/Java_R1_Experiment2_Read_times.dat',
                    f'{root_path}/read_write/Java_R2_Experiment2_Read_times.dat',
                    f'{root_path}/read_write/Java_R3_Experiment2_Read_times.dat',
                    f'{root_path}/read_write/Rust_R13_Experiment2_Read_times.dat']                 


   
    load_to_memory_results = [f'{root_path}/read_write/CPP_R13_Experiment1_LoadToMemory_times.dat',
                              f'{root_path}/read_write/Java_R1_Experiment1_LoadToMemory_times.dat',
                              f'{root_path}/read_write/Java_R2_Experiment1_LoadToMemory_times.dat',
                              f'{root_path}/read_write/Java_R3_Experiment1_LoadToMemory_times.dat',
                              f'{root_path}/read_write/Rust_R13_Experiment1_LoadToMemory_times.dat']
   
    read_write_header = ["baseline","language","taskset","execution","platform","seq_rand","nrow","time"]   
    load_to_memory_header = ["language","platform","taskset","nrow","time"]

    baselines = ["DefaultJava","Json+GzipJava","BsonJava","ProtoBufJava","KryoJava","ByteBufferJava","JsonJava","FlatBuffersJava",
                "HandcodedCPP","inPlaceCPP","BoostCPP","ProtoBufCPP","BsonCPP","FlatBufCPP",
                "JsonRust","BincodeRust","MessagePackRust","BsonRust","FlexBufRust"]    
    
    nrows = [10000000,9000000,8000000,7000000,6000000,5000000,4000000,3000000,2000000,1000000]

    executions = ["CPU", "IO", "Total"]

    platforms = ["Single"] #, "Parallel" 

    seq_rands = ["Sequential"] #, "Random"

    languages = ["Java", "CPP","Rust"]

    tasksets = [True, False]

    df_write = merge_results(write_results)
    df_read = merge_results(read_results)
    df_load_to_memory = merge_results(load_to_memory_results)

    # write    
    df_write_final = pd.DataFrame(columns = read_write_header)    
    for baseline in baselines:
        for language in languages:
            for taskset in tasksets:
                #for execution in executions:
                     for platform in platforms:
                         for seq_rand in seq_rands:
                             for nrow in nrows:                                                                      
                                df_total =  df_write.loc[(df_write['baseline']==baseline) &
                                                    (df_write["language"]==language) &
                                                    (df_write['taskset']==taskset) &
                                                    (df_write['execution']=='Total') &
                                                    (df_write['platform']==platform) &
                                                    (df_write['seq_rand']==seq_rand) &
                                                    (df_write['nrow']==nrow)
                                                    ]                                                    
                                list_total = np.array(df_total['time'].tolist())  

                                df_cpu =  df_write.loc[(df_write['baseline']==baseline) &
                                                    (df_write["language"]==language) &
                                                    (df_write['taskset']==taskset) &
                                                    (df_write['execution']=='CPU') &
                                                    (df_write['platform']==platform) &
                                                    (df_write['seq_rand']==seq_rand) &
                                                    (df_write['nrow']==nrow)
                                                    ]                                                    
                                list_cpu = np.array(df_cpu['time'].tolist())  

                                df_load =  df_load_to_memory.loc[(df_load_to_memory["language"]==language) &
                                                                 (df_load_to_memory['taskset']==taskset) &                                                                                                                    
                                                                 (df_load_to_memory['platform']==platform) &
                                                                 (df_load_to_memory['nrow']==nrow)
                                                    ]                                                    
                                list_load = np.array(df_load['time'].tolist())  


                                if len(list_total) > 0:
                                    
                                    time_total = calc_mean(list=list_total)    
                                    time_cpu = calc_mean(list=list_cpu)
                                    time_load = calc_mean(list=list_load)
                                    #print(f'count={nrow} platform={platform} execution={seq_rand} taskset={taskset} locad time {baseline} = {time_load} ')
                                    tmptime1  = time_total - time_cpu
                                    tmptime2  = time_cpu - time_load
                                    

                                    base = baseline.replace("FlatBuffersJava", "FlatBufJava")                                    
                                    result_total = [base,language,taskset,'Total',platform,seq_rand,nrow,f'{(time_total - time_load):.2f}']
                                    result_cpu = [base,language,taskset,'CPU',platform,seq_rand,nrow,f'{(time_cpu - time_load):.2f}']
                                    result_io = [base,language,taskset,"IO",platform,seq_rand,nrow,f'{tmptime1:.2f}']

                                    # if(tmptime1 <0):

                                    #     print(f'load time = {time_load}')    
                                    #     # print(result_cpu)
                                    #     # print(f'cpus = {list_cpu}   load={list_load}')
                                    #     #print("----------------")
                                    #     print(f'total={time_total}  cpus = {list_cpu}   load={list_load}')
                                    #     print(result_io)
                                    #     #tmptime = 2
                                    #     print("+++++++++++++++++++++++++++++")

                                    # if time_total - time_cpu < 0 :
                                    #     print(result_total)
                                    #     print(result_io)
                                    #     print(f'{taskset}, {baseline}, {nrow}, [{list_total}], [{list_cpu}], {seq_rand}')
                                    #     print("++++++++++++++++++++++++++++++")
                                                                        
                                    df_write_final.loc[len(df_write_final)] = result_total
                                    df_write_final.loc[len(df_write_final)] = result_cpu
                                    df_write_final.loc[len(df_write_final)] = result_io



  
    df_write_final.to_csv("write.csv", index=False)      
    ################################################
    # Read 
    #                                                    
    df_read_final = pd.DataFrame(columns = read_write_header)    
    for baseline in baselines:
        for language in languages:
            for taskset in tasksets:
                     for platform in platforms:
                         for seq_rand in seq_rands:
                             for nrow in nrows:                                                                      
                                df_total =  df_read.loc[(df_read['baseline']==baseline) &
                                                    (df_read["language"]==language) &
                                                    (df_read['taskset']==taskset) &
                                                    (df_read['execution']=='Total') &
                                                    (df_read['platform']==platform) &
                                                    (df_read['seq_rand']==seq_rand) &
                                                    (df_read['nrow']==nrow)
                                                    ]                                                    
                                list_total = np.array(df_total['time'].tolist())  

                                df_io =  df_read.loc[(df_read['baseline']==baseline) &
                                                    (df_read["language"]==language) &
                                                    (df_read['taskset']==taskset) &
                                                    (df_read['execution']=='IO') &
                                                    (df_read['platform']==platform) &
                                                    (df_read['seq_rand']==seq_rand) &
                                                    (df_read['nrow']==nrow)
                                                    ]                                                    
                                list_io = np.array(df_io['time'].tolist())                                  


                                if len(list_total) > 0:
                                    
                                    time_total = calc_mean(list=list_total)    
                                    time_io = calc_mean(list=list_io)
                                    
                                    base = baseline.replace("FlatBuffersJava", "FlatBufJava")                                    
                                    result_total = [base,language,taskset,'Total',platform,seq_rand,nrow,f'{time_total:.2f}']
                                    result_cpu = [base,language,taskset,'CPU',platform,seq_rand,nrow,f'{(time_total - time_io):.2f}']
                                    result_io = [base,language,taskset,"IO",platform,seq_rand,nrow,f'{time_io:.2f}']
                                                                        
                                    df_read_final.loc[len(df_read_final)] = result_total
                                    df_read_final.loc[len(df_read_final)] = result_cpu
                                    df_read_final.loc[len(df_read_final)] = result_io



  
    df_read_final.to_csv("read.csv", index=False)                  
  


   

In [6]:
results()